In [2]:
!pip install emoji

In [3]:
import numpy as np
import pandas as pd
import emoji

from keras.models import Sequential
from keras.layers import Dense, Input, Dropout, SimpleRNN,LSTM, Activation
from keras.utils import np_utils

import matplotlib.pyplot as plt

In [4]:
train = pd.read_csv('train_emoji.csv',header= None)
test = pd.read_csv('test_emoji.csv',header= None)
train.drop(labels= [2, 3], axis= 1, inplace= True)

In [5]:
test.head()

,0,1
0,I want to eat,4
1,he did not answer,3
2,he got a raise,2
3,she got me a present,0
4,ha ha ha it was so funny,2


In [6]:
emoji.EMOJI_UNICODE_ENGLISH

{':1st_place_medal:': '🥇',
 ':2nd_place_medal:': '🥈',
 ':3rd_place_medal:': '🥉',
 ':AB_button_(blood_type):': '🆎',
 ':ATM_sign:': '🏧',
 ':A_button_(blood_type):': '🅰️',
 ':Afghanistan:': '🇦🇫',
 ':Albania:': '🇦🇱',
 ':Algeria:': '🇩🇿',
 ':American_Samoa:': '🇦🇸',
 ':Andorra:': '🇦🇩',
 ':Angola:': '🇦🇴',
 ':Anguilla:': '🇦🇮',
 ':Antarctica:': '🇦🇶',
 ':Antigua_&_Barbuda:': '🇦🇬',
 ':Aquarius:': '♒',
 ':Argentina:': '🇦🇷',
 ':Aries:': '♈',
 ':Armenia:': '🇦🇲',
 ':Aruba:': '🇦🇼',
 ':Ascension_Island:': '🇦🇨',
 ':Australia:': '🇦🇺',
 ':Austria:': '🇦🇹',
 ':Azerbaijan:': '🇦🇿',
 ':BACK_arrow:': '🔙',
 ':B_button_(blood_type):': '🅱️',
 ':Bahamas:': '🇧🇸',
 ':Bahrain:': '🇧🇭',
 ':Bangladesh:': '🇧🇩',
 ':Barbados:': '🇧🇧',
 ':Belarus:': '🇧🇾',
 ':Belgium:': '🇧🇪',
 ':Belize:': '🇧🇿',
 ':Benin:': '🇧🇯',
 ':Bermuda:': '🇧🇲',
 ':Bhutan:': '🇧🇹',
 ':Bolivia:': '🇧🇴',
 ':Bosnia_&_Herzegovina:': '🇧🇦',
 ':Botswana:': '🇧🇼',
 ':Bouvet_Island:': '🇧🇻',
 ':Brazil:': '🇧🇷',
 ':British_Indian_Ocean_Territory:': '🇮🇴',
 ':British_Virgin_

In [7]:
emoji.EMOJI_UNICODE_ENGLISH[':crying_face:']

'😢'

In [8]:
emoji_dict = {
    "0":':face_blowing_a_kiss:',
    "1":':basketball:',
    "2":':face_with_open_mouth:',
    "3":':face_with_head-bandage:',
    "4":':banana:'
}

In [9]:
for item in emoji_dict.values():
  print(emoji.emojize(item), end= '')

😘🏀😮🤕🍌

#Text Pre Processing


In [10]:
 x_train = train[0].values
 y_train = train[1].values
 x_test = train[0].values
 y_test = train[1].values

In [11]:
x_train[:5]

array(['never talk to me again', 'I am proud of your achievements',
       'It is the worst day in my life', 'Miss you so much',
       'food is life'], dtype=object)

In [12]:
y_train[:5]

array([3, 2, 3, 0, 4])

In [13]:
x_train.shape , y_train.shape

((132,), (132,))

##Get embeddings


In [14]:
embeddings_matrix = {}

In [15]:
f = open('glove.6B.50d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_matrix[word] = coefs
f.close()

In [16]:
embeddings_matrix['the'].shape

(50,)

In [17]:
def get_embedding_matrix_for_data(data):
  maxLen = int(10)
  embedding_data = np.zeros((len(data), maxLen, 50))

  for i in range(data.shape[0]):
    words_in_sentence = data[i].split()

    for j in range(len(words_in_sentence)):
      if embeddings_matrix.get(words_in_sentence[j].lower()) is not None:
        embedding_data[i][j] = embeddings_matrix[words_in_sentence[j].lower()]

  return embedding_data  

In [18]:
x_train_embed = get_embedding_matrix_for_data(x_train)
x_test_embed = get_embedding_matrix_for_data(x_test)
print(x_train_embed.shape)
print(x_test_embed.shape)

(132, 10, 50)
(132, 10, 50)


converting output to categorical

In [19]:
from keras.utils.np_utils import to_categorical

In [20]:
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)
print(y_train)
print(y_test)

[[0. 0. 0. 1. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0.]
 [1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0.]
 [1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 1.]
 [0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1.]
 [0. 0. 1. 0. 0.]
 [1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1.]
 [0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0.]
 [1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0.]
 [1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 1. 0.]
 [1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1.]
 [0. 0. 1.

#Model  Building


In [21]:
import keras

In [22]:
model = Sequential()
model.add(LSTM(units = 64, input_shape = (10, 50), return_sequences= True))
model.add(Dropout(0.3))
model.add(LSTM(units= 32))
model.add(Dropout(0.3))
model.add(Dense(units= 10, activation= 'relu'))
model.add(Dense(units= 5, activation='softmax'))

In [23]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 10, 64)            29440     
                                                                 
 dropout (Dropout)           (None, 10, 64)            0         
                                                                 
 lstm_1 (LSTM)               (None, 32)                12416     
                                                                 
 dropout_1 (Dropout)         (None, 32)                0         
                                                                 
 dense (Dense)               (None, 10)                330       
                                                                 
 dense_1 (Dense)             (None, 5)                 55        
                                                                 
Total params: 42,241
Trainable params: 42,241
Non-traina

In [24]:
model.compile(optimizer='adam', loss = keras.losses.categorical_crossentropy, metrics=['acc'])

In [25]:
hstry = model.fit(x_train_embed, y_train, validation_split=0.1, batch_size=32, epochs=50)

Epoch 1/50
4/4 [==============================] - 8s 355ms/step - loss: 1.5939 - acc: 0.3136 - val_loss: 1.6331 - val_acc: 0.0000e+00
Epoch 2/50
4/4 [==============================] - 0s 19ms/step - loss: 1.5684 - acc: 0.2966 - val_loss: 1.6801 - val_acc: 0.0714
Epoch 3/50
4/4 [==============================] - 0s 24ms/step - loss: 1.5531 - acc: 0.3305 - val_loss: 1.7082 - val_acc: 0.0714
Epoch 4/50
4/4 [==============================] - 0s 19ms/step - loss: 1.5320 - acc: 0.3475 - val_loss: 1.6904 - val_acc: 0.0714
Epoch 5/50
4/4 [==============================] - 0s 19ms/step - loss: 1.5274 - acc: 0.3220 - val_loss: 1.6737 - val_acc: 0.0714
Epoch 6/50
4/4 [==============================] - 0s 18ms/step - loss: 1.4989 - acc: 0.3051 - val_loss: 1.6556 - val_acc: 0.0714
Epoch 7/50
4/4 [==============================] - 0s 19ms/step - loss: 1.4741 - acc: 0.3220 - val_loss: 1.6356 - val_acc: 0.0714
Epoch 8/50
4/4 [==============================] - 0s 19ms/step - loss: 1.4662 - acc: 0.3136 

In [26]:
model.evaluate(x_train_embed, y_train)[1]*100

5/5 [==============================] - 0s 6ms/step - loss: 0.4011 - acc: 0.8864


88.63636255264282

In [27]:
model.evaluate(x_test_embed, y_test)[1]*100

5/5 [==============================] - 0s 6ms/step - loss: 0.4011 - acc: 0.8864


88.63636255264282

In [28]:
pred = model.predict(x_test_embed)
y_pred=np.argmax(pred,axis=1)

In [29]:
for t in range(len(test)):
  print(test[0].iloc[t])
  print("predictions: ", emoji.emojize(emoji_dict[str(y_pred[t])]))
  print("Actual: ", emoji.emojize(emoji_dict[str(test[1].iloc[t])]))

I want to eat
predictions:  🤕
Actual:  🍌
he did not answer
predictions:  😮
Actual:  🤕
he got a raise
predictions:  🤕
Actual:  😮
she got me a present
predictions:  😘
Actual:  😘
ha ha ha it was so funny
predictions:  🍌
Actual:  😮
he is a good friend
predictions:  😘
Actual:  😘
I am upset
predictions:  🤕
Actual:  😘
We had such a lovely dinner tonight
predictions:  😮
Actual:  😘
where is the food
predictions:  🤕
Actual:  🍌
Stop making this joke ha ha ha
predictions:  🏀
Actual:  😮
where is the ball
predictions:  🤕
Actual:  🏀
Work is hard
predictions:  🤕
Actual:  🤕
This girl is messing with me
predictions:  🏀
Actual:  🤕
are you serious ha ha
predictions:  🤕
Actual:  😮
Let us go play baseball
predictions:  😮
Actual:  🏀
This stupid grader is not working
predictions:  🤕
Actual:  🤕
work is horrible
predictions:  😮
Actual:  🤕
Congratulation for having a baby
predictions:  🤕
Actual:  😮
stop messing around
predictions:  🏀
Actual:  🤕
any suggestions for dinner
predictions:  😮
Actual:  🍌
I love taking 